In [1]:
import os, sys, glob
import logging
import numpy as np
import matplotlib
from matplotlib.colors import LogNorm
from matplotlib import pyplot as plt

from scipy.optimize import minimize

import numba

import utils
from signal_generator import SignalGenerator
from background_generator import BackgroundGenerator

In [2]:
def calculate_ns(sob,epsilon=1e-4,max_iter=10):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> Df = lambda x: 2*x - 1
    >>> newton(f,Df,1,1e-8,10)
    Found solution after 5 iterations.
    1.618033988749989
    '''
    xn = 0
    k = 1/(sob-1)
    f = lambda x: (1/(x+k)).sum()
    Df = lambda x: -(1/(x+k)**2).sum()
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon: return xn
        Dfxn = Df(xn)
        if Dfxn == 0: return 0
        xn = xn - fxn/Dfxn
        if xn < 0: xn=0
        if xn > 1: xn=1
    return xn



In [3]:
class pointsource_analysis:
    def __init__(self, angular_window_deg=10):
        public_data_directory = "../gxn/gXn_simulations/icecube_10year_ps/"

        aeffs = sorted(glob.glob(os.path.join(public_data_directory, 
                                              "irfs/IC86_II_effectiveArea.csv")))
        smearing = sorted(glob.glob(os.path.join(public_data_directory, 
                                                 "irfs/IC86_II_smearing.csv")))
        uptime = [os.path.join(public_data_directory, "uptime/IC86_II_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_III_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_IV_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_V_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_VI_exp.csv"),
                  os.path.join(public_data_directory, "uptime/IC86_VII_exp.csv")]
        data = [os.path.join(public_data_directory, "events/IC86_II_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_III_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_IV_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_V_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_VI_exp.csv"),
                os.path.join(public_data_directory, "events/IC86_VII_exp.csv")]

        self.sig_gen = SignalGenerator(aeffs, smearing, uptime,
                            angular_resolution_scale=1.0)
        self.bg_gen = BackgroundGenerator(data)
        self.angular_window_deg = angular_window_deg

        return
    
    def one_trial(self, dec_deg ,ra_deg, integrated_flux):
        signal = self.sig_gen.generate_pointsource(declination_deg = dec_deg, 
                                    right_ascension_deg = ra_deg,
                                    integral_flux_function = integrated_flux)
        background = self.bg_gen.scramble()
        return (np.concatenate((background.to_records(),
                               signal.to_records())), len(signal))
        
    def _trim(self, events, dec_deg, ra_deg, angle_deg=10):
        dist = utils.ang_dist(np.radians(ra_deg),
                      np.radians(dec_deg),
                      events['ra'], events['dec'])
        return events[dist < np.radians(angle_deg)]
    
    def calculate_sob(self, events, dec_deg, ra_deg, integrated_flux):
        S = self.sig_gen.pdf_pointsource(events,
                                          declination_deg = dec_deg, 
                                          right_ascension_deg = ra_deg,
                                          integral_flux_function = integrated_flux)
        Snllh = np.log(S)
        Bnllh = self.bg_gen.logpdf(events['logE'], np.sin(events['dec'])) - np.log(2*np.pi) 
        sob = Snllh-Bnllh 
        sob[sob == -np.inf] = sob[sob > -np.inf].min()
        sob[~np.isfinite(sob)] = sob[np.isfinite(sob)].max()
        return sob
    
    def fit(self, events, dec_deg, ra_deg, integrated_flux):
        nevents = len(events)
        events = self._trim(events, dec_deg, ra_deg, self.angular_window_deg)
        def ts(x, return_ns=False):
            if len(events) == 0: return 0
            ns, gamma = x
            f = lambda emin, emax: integrated_flux(emin, emax, 
                                                   normalization = 1,
                                                   index = gamma)
            sob = np.exp(self.calculate_sob(events, dec_deg, ra_deg, f))
            sob = sob[sob>0]
            #ns = calculate_ns(sob)
            llh = np.log(ns * (sob-1) + 1)
            if return_ns: 
                return -llh.sum(), ns * nevents
            else:
                return -llh.sum()
        
        results = minimize(ts, [0, -2], method='SLSQP',
                           bounds = [(0, nevents), (-4, -1)])
        
        # get the ns back out
        #llh, ns = ts(results.x, True)
        
        return results.x, results.fun
    

In [4]:
x = pointsource_analysis(angular_window_deg=10)

/data/condor_builds/users/mlarson/public_data_trials/signal_generator.py:90: RuntimeWarning: divide by zero encountered in log
  - np.cos(np.radians(smearing['AngErr_max[deg]'])))))


In [8]:
def powerlaw_integral_flux(emin, emax, normalization=1e-15, E0=1000, index=-2):
    if index > 0:
        logging.error("Please give a spectral index below 0")
        sys.exit(1)

    integral = normalization/E0**index
    if index == -1:  integral *= np.log(emax/emin)
    elif index != -1:
        integral *= (emax**(1+index) - emin**(1+index))/(1+index)
    return integral
dec_deg, ra_deg = 0, 10

gen_args = [dec_deg, ra_deg, powerlaw_integral_flux]
fit_args = [dec_deg, ra_deg, powerlaw_integral_flux]

# Produce a trial!
events, ninj = x.one_trial(*gen_args)

# Fit it!
print(ninj, x.fit(events, *fit_args))

/data/i3home/mjlarson/.local/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/data/i3home/mjlarson/.local/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in log


ValueError: zero-size array to reduction operation maximum which has no identity

In [220]:
ntrials = 10
ninj, ns = [], []


for flux in np.logspace(-15, -13, 11):
    print('flux:', flux)
    def powerlaw_integral_flux(emin, emax, normalization=flux, E0=1000, index=-2):
        if index > 0:
            logging.error("Please give a spectral index below 0")
            sys.exit(1)

        integral = normalization/E0**index
        if index == -1:  integral *= np.log(emax/emin)
        elif index != -1:
            integral *= (emax**(1+index) - emin**(1+index))/(1+index)
        return integral
    dec_deg, ra_deg = 0, 10

    gen_args = [dec_deg, ra_deg, powerlaw_integral_flux]
    fit_args = [dec_deg, ra_deg, powerlaw_integral_flux]

    out = np.zeros(ntrials, dtype=float)
    for trial in range(ntrials):
        print(trial, end=', ')
        # Produce a trial!
        events, ni = x.one_trial(*gen_args)

        # Fit it!
        values, ts = x.fit(events, *fit_args)
        ninj.append(ni)
        ns.append(values[0])
                

flux: 1e-15
0, 

/data/i3home/mjlarson/.local/lib/python3.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning: divide by zero encountered in log
/data/i3home/mjlarson/.local/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in log


ValueError: zero-size array to reduction operation maximum which has no identity